## Open API : APT Price Seoul

- 지역코드와 거래년월을 입력하면 해당 거래 데이터를 수집하는 사용자 정의 함수를 생성합니다.
- 지역코드와 거래년월을 변경하면 원하는 데이터를 빠르게 수집할 수 있습니다.

### 아파트 거래 데이터 수집하는 사용자 정의 함수 생성

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests
import xmltodict
import pandas as pd

# 지역코드와 거래년월을 입력하면 데이터를 수집하는 사용자 정의 함수를 생성합니다.
def aptPrice(areaCd, ymonth, apiKey):
    
    # 요청 URL을 설정합니다.
    url = 'http://openapi.molit.go.kr:8081'
    path = '/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc'
    oper = '/getRTMSDataSvcAptTrade'
    
    # 쿼리 문자열을 딕셔너리로 설정합니다.
    query = {
        'LAWD_CD': areaCd, 
        'DEAL_YMD': ymonth, 
        'serviceKey': apiKey
    }
    
    # HTTP 요청을 실행합니다.
    res = requests.get(url = url + path + oper, params = query)
    
    # HTTP 응답 바디 문자열을 딕셔너리로 변환하여 data에 할당합니다.
    data = xmltodict.parse(xml_input = res.text)
    
    # 수집할 데이터(딕셔너리)로 데이터프레임을 생성합니다.
    df = pd.DataFrame(data = data['response']['body']['items']['item'])
    
    # df을 반환합니다.
    return df

In [ ]:
# 공공데이터 포털 오픈 API 인증키를 설정합니다.
apiKey = '공공데이터포털 일반 인증키(Decoding) 붙여넣기'

In [ ]:
# 함수를 테스트합니다.
apt = aptPrice(areaCd = '11680', ymonth = '202211', apiKey = apiKey)

In [ ]:
# apt의 정보를 확인합니다.
apt.info()

In [ ]:
# apt의 처음 5행을 출력합니다.
apt.head()

### 작업 경로 확인 및 변경

In [ ]:
# 관련 라이브러리를 호출합니다.
import os

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 작업 경로에 있는 폴더명과 파일명을 출력합니다.
os.listdir()

### 지역코드 준비

In [ ]:
# 서울특별시 자치구 정보가 포함된 엑셀파일을 읽습니다.
dong = pd.read_excel(io = 'Area_Code.xlsx')

In [ ]:
# 데이터프레임의 정보를 확인합니다.
dong.info()

In [ ]:
# dong의 처음 10행을 출력합니다.
dong.head(n = 10)

In [ ]:
# code를 문자열로 변환합니다.
dong['code'] = dong['code'].astype('str')

### 반복문 실행

In [ ]:
# 관련 라이브러리를 호출합니다.
# from tqdm.notebook import tqdm
from tqdm import tqdm
import time

In [ ]:
# 최종 결과를 저장할 빈 데이터프레임을 생성합니다.
apt = pd.DataFrame()

In [ ]:
# 반복문을 실행할 범위를 설정합니다.
n = len(dong)

In [ ]:
# 거래년월을 설정합니다.
ymonth = '202201'

In [ ]:
# 반복문을 실행합니다.
for i in tqdm(range(n)):
    
    # 지역명을 설정합니다.
    areaNm = dong['sido'][i] + ' ' + dong['sigg'][i]
    
    # 지역코드는 code의 처음 5글자를 잘라낸 값으로 설정합니다.
    areaCd = dong['code'][i][0:5]
    
    # 거래 건수가 없으면 에러를 발생하고 반복문이 중단됩니다.
    # 따라서 예외 처리를 추가합니다.
    try:
        
        # 서울특별시 자치구 단위로 거래 데이터를 수집합니다.
        df = aptPrice(areaCd = areaCd, ymonth = ymonth, apiKey = apiKey)
        
        # 필요한 열을 추가합니다.
        df['시도'] = dong['sido'][i]
        df['시군구'] = dong['sigg'][i]
        
        # 최종 결과 데이터프레임에 apt를 추가합니다.
        # [참고] 인덱스가 중복되면 에러를 반환합니다.
        apt = pd.concat(objs = [apt, df], ignore_index = True)
    
    except:
        print('> 거래 건수가 없으므로 다음 자치구로 이동합니다!')
        next
    
    # 1초 간 멈춥니다.
    time.sleep(1)

In [ ]:
# apt의 정보를 확인합니다.
apt.info()

In [ ]:
# 거래금액에서 콤마를 삭제하고 정수형으로 변환합니다.
apt['거래금액'] = apt['거래금액'].str.replace(',', '').astype('int')

In [ ]:
# 서울특별시 자치구별 아파트매매 건수를 확인합니다.
apt['시군구'].value_counts().sort_index()

In [ ]:
# 서울특별시 자치구별 아파트매매 평균 가격을 확인합니다.
apt.groupby(by = '시군구')['거래금액'].mean().round().astype('int')

### xlsx 파일로 저장

In [ ]:
# xlsx 파일명을 생성합니다.
fileName = f'APT_Price_Seoul_{ymonth}.xlsx'

In [ ]:
# xlsx 파일로 저장합니다.
apt.to_excel(excel_writer = fileName, index = False)

## End of Document